In [2]:
import cv2
import os
import random
import numpy as np
import time

**This function is for converting the video to frames while testing for new video data**

In [ ]:
# def video_to_frames(video_path, output_parent_folder, num_frames=50):
#   start_time = time.time()
#   # Create output folder if it doesn't exist
#   os.makedirs(output_parent_folder, exist_ok=True)

#   # Get the video name (excluding extension) from the video path
#   video_name = os.path.splitext(os.path.basename(video_path))[0]

#   # Create a folder with the video name inside the output parent folder
#   output_folder = os.path.join(output_parent_folder, video_name)
#   os.makedirs(output_folder, exist_ok=True)

#   # Open the video file
#   cap = cv2.VideoCapture(video_path)

#   # Get the frames per second (fps) and frame count
#   fps = int(cap.get(cv2.CAP_PROP_FPS)) # gets the frames per second (fps) of the video. FPS indicates how many frames are displayed per second in the video.
#   frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # retrieves the total number of frames in the video

#   print(f"Video FPS: {fps}")
#   print(f"Total Frames: {frame_count}")

#   # randomly select num_frames frame
#   selected_frames = random.sample(range(frame_count), min(num_frames, frame_count))

#   # # serially select num_frames frame
#   # selected_frames = list(range(0, min(num_frames, frame_count)))

#   # Read and save each selected frame
#   for frame_number in selected_frames:
#     # Set the video capture object to the selected frame
#     cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

#     # Read the selected frame
#     success, frame = cap.read()

#     if success:
#       # Resize the frame to (224, 224)
#       frame = cv2.resize(frame, (224, 224))

#       # Save the frame as an image file
#       frame_filename = f"frame_{frame_number:04d}.jpg"
#       frame_path = os.path.join(output_folder, frame_filename)
#       cv2.imwrite(frame_path, frame)

#   # Release the video capture object
#   cap.release()

#   end_time = time.time()
#   difference = end_time - start_time
#   print(f"frame extraction time: {difference} seconds")

#   print(f"{num_frames} frames randomly selected and saved successfully!")
#   print('\n')

# video_path = "/content/drive/MyDrive/dataset_new"
# output_parent_folder = "/content/drive/MyDrive/new_train_feature"
# video_to_frames(video_path, output_parent_folder)


### MediaPipe implementation

In [ ]:
def video_to_frames(video_path, output_parent_folder, num_frames=50):
    start_time = time.time()
    # Create output folder if it doesn't exist
    os.makedirs(output_parent_folder, exist_ok=True)

    # Get the video name (excluding extension) from the video path
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Create a folder with the video name inside the output parent folder
    output_folder = os.path.join(output_parent_folder, video_name)
    os.makedirs(output_folder, exist_ok=True)

    # Initialize MediaPipe Holistic model
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the frames per second (fps) and frame count
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video FPS: {fps}")
    print(f"Total Frames: {frame_count}")

    # Randomly select num_frames frame
    # selected_frames = random.sample(range(frame_count), min(num_frames, frame_count))

    # Serially select num_frames frame
    selected_frames = list(range(0, min(num_frames, frame_count)))


    # Read and save each selected frame
    for frame_number in selected_frames:
        # Set the video capture object to the selected frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # Read the selected frame
        success, frame = cap.read()

        if success:
            # Convert the frame to RGB for processing with MediaPipe
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Perform holistic detection
            results = holistic.process(frame_rgb)

            # Draw bounding box around detected bodies
            if results.pose_landmarks:
                # Extract landmark coordinates
                landmarks = [(lm.x * frame.shape[1], lm.y * frame.shape[0]) for lm in results.pose_landmarks.landmark]

                # Compute bounding box coordinates
                x_min = min(landmark[0] for landmark in landmarks)
                x_max = max(landmark[0] for landmark in landmarks)
                y_min = min(landmark[1] for landmark in landmarks)
                y_max = max(landmark[1] for landmark in landmarks)

                # Crop the frame according to the bounding box
                cropped_image = frame[int(y_min):int(y_max), int(x_min):int(x_max)]

                # Resize the cropped frame to (224, 224)
                cropped_image_resized = cv2.resize(cropped_image, (224, 224))

                # Save the cropped and resized frame as an image file
                frame_filename = f"frame_{frame_number:04d}.jpg"
                frame_path = os.path.join(output_folder, frame_filename)
                cv2.imwrite(frame_path, cropped_image_resized)

    # Release the video capture object
    cap.release()

    end_time = time.time()
    difference = end_time - start_time
    print(f"Frame extraction time: {difference} seconds")

    print(f"{num_frames} frames randomly selected, cropped, and saved successfully!")
    print('\n')

    # Release the holistic model
    holistic.close()

# Example usage
video_path = "/content/drive/MyDrive/dataset_new/video.mp4"
output_parent_folder = "/content/drive/MyDrive/new_train_feature"
video_to_frames(video_path, output_parent_folder)


**converting video to frames, selecting and resizeing randomly 80 frames of the video**

In [2]:
def process_videos(source_folder, destination_folder, num_frames_to_select=50, resize_dimensions=(224, 224)):
  # Create output folder if it doesn't exist
  os.makedirs(destination_folder, exist_ok=True)

  # List all ffiles in the source folder
  video_files = [file for file in os.listdir(source_folder) if file.endswith('.mp4')]

  for video_file in video_files:
    # Create a folder for each video inside the destination folder
    video_name = os.path.splitext(video_file)[0]
    output_video_folder = os.path.join(destination_folder, video_name)
    os.makedirs(output_video_folder, exist_ok=True)

    # Open the video file
    video_path = os.path.join(source_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    # Get the video FPS andtotal number of frames
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video: {video_file}")
    print(f"FPS: {fps}")
    print(f"Total Frames: {total_frames}")

    # Select 50 random frames
    # selected_frames = random.sample(range(total_frames), min(num_frames_to_select, total_frames))

    # Serially select num_frames frame
    selected_frames = list(range(0, min(num_frames_to_select, total_frames)))


    start_time = time.time()

    # Read and process each selected frame
    for frame_number in selected_frames:
      # Set the video capture object to the selected frame
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

      # Read the selected frame
      success, frame = cap.read()

      if success:
        # Resize the frame
        resized_frame = cv2.resize(frame, resize_dimensions)

        # Save the resized frame as an image file
        frame_filename = f"frame_{frame_number:04d}.jpg"
        frame_path=os.path.join(output_video_folder, frame_filename)
        cv2.imwrite(frame_path, resized_frame)

    # Release the video capture object
    cap.release()

    print(f"{num_frames_to_select} frames randomly selected and resized for video : {video_file}.")

    end_time = time.time()
    difference = end_time - start_time

    print(f"Time Taken =: {difference} seconds")
    print('\n')

source_folder = "/content/drive/MyDrive/latest_dataset"
destination_folder = "/content/drive/MyDrive/sequence_train_data"
num_frames_to_select= 50
resize_dimensions = (224, 224)

process_videos(source_folder, destination_folder, num_frames_to_select, resize_dimensions)

Video: video197.mp4
FPS: 30
Total Frames: 84
50 frames randomly selected and resized for video : video197.mp4.
Time Taken =: 61.207719564437866 seconds


Video: video209.mp4
FPS: 120
Total Frames: 163


KeyboardInterrupt: 

** Checking the shape of feature of video**

In [3]:
def check_file_shape(folder_path, target_shape=(50, 4096)):
  # List all files in the folder
  file_names = [file for file in os.listdir(folder_path) if file.endswith('.npy')]

  for file_name in file_names:
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)

    # Load the numpy array from the file
    try:
      data = np.load(file_path)
      # Check if the shape matches the target shape
      if data.shape != target_shape:
        print(f"Warning: File {file_name} does not have the expected shape. Actual shape: {data.shape}")
      else:
        print(f"File {file_name} has the expected shape")
        print(f"Actual shape: {data.shape}")

    except Exception as e:
      print(f"Error loading file {file_name}: {str(e)}")

folder_path = "/content/drive/MyDrive/sequence_train_feature"
target_shape = (50, 4096)

check_file_shape(folder_path, target_shape)

File video197.npy has the expected shape
Actual shape: (50, 4096)
File video209.npy has the expected shape
Actual shape: (50, 4096)
File video218.npy has the expected shape
Actual shape: (50, 4096)
File video207.npy has the expected shape
Actual shape: (50, 4096)
File video198.npy has the expected shape
Actual shape: (50, 4096)
File video210.npy has the expected shape
Actual shape: (50, 4096)
File video193.npy has the expected shape
Actual shape: (50, 4096)
File video215.npy has the expected shape
Actual shape: (50, 4096)
File video196.npy has the expected shape
Actual shape: (50, 4096)
File video208.npy has the expected shape
Actual shape: (50, 4096)
File video190.npy has the expected shape
Actual shape: (50, 4096)
File video235.npy has the expected shape
Actual shape: (50, 4096)
File video237.npy has the expected shape
Actual shape: (50, 4096)
File video195.npy has the expected shape
Actual shape: (50, 4096)
File video221.npy has the expected shape
Actual shape: (50, 4096)
File video

In [3]:
!pip3 install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.8/34.8 MB 26.3 MB/s eta 0:00:00


In [4]:
import os
import cv2
import random
import time
import mediapipe as mp

# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

def process_videos(source_folder, destination_folder, num_frames_to_select=50, resize_dimensions=(224, 224)):
    # Create output folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # List all files in the source folder
    video_files = [file for file in os.listdir(source_folder) if file.endswith(('.mov', '.mp4'))]

    for video_file in video_files:
        # Create a folder for each video inside the destination folder
        video_name = os.path.splitext(video_file)[0]
        output_video_folder = os.path.join(destination_folder, video_name)
        os.makedirs(output_video_folder, exist_ok=True)

        # Open the video file
        video_path = os.path.join(source_folder, video_file)
        cap = cv2.VideoCapture(video_path)

        # Get the video FPS and total number of frames
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"Video: {video_file}")
        print(f"FPS: {fps}")
        print(f"Total Frames: {total_frames}")

        # Select 50 random frames
        # selected_frames = random.sample(range(total_frames), min(num_frames_to_select, total_frames))

        # Serially select num_frames frame
        selected_frames = list(range(0, min(num_frames_to_select, total_frames)))

        start_time = time.time()

        # Read and process each selected frame
        for frame_number in selected_frames:
            # Set the video capture object to the selected frame
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

            # Read the selected frame
            success, frame = cap.read()

            if success:
                # Convert the image to RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Perform holistic detection
                results = holistic.process(frame_rgb)

                # Draw bounding box around detected bodies
                if results.pose_landmarks:
                    # Extract landmark coordinates
                    landmarks = [(lm.x * frame.shape[1], lm.y * frame.shape[0]) for lm in results.pose_landmarks.landmark]

                    # Compute bounding box coordinates with additional width and height
                    x_min = min(landmark[0] for landmark in landmarks)
                    x_max = max(landmark[0] for landmark in landmarks)
                    y_min = min(landmark[1] for landmark in landmarks)
                    y_max = max(landmark[1] for landmark in landmarks)

                    # Add more padding to top, left, and right
                    padding_top = 0.3  # Adjust as needed
                    padding_bottom = 0.1  # Adjust as needed
                    padding_left = 0.2  # Adjust as needed
                    padding_right = 0.2  # Adjust as needed

                    width = x_max - x_min
                    height = y_max - y_min
                    y_min = max(0, int(y_min - padding_top * height))
                    y_max = min(frame.shape[0], int(y_max + padding_bottom * height))
                    x_min = max(0, int(x_min - padding_left * width))
                    x_max = min(frame.shape[1], int(x_max + padding_right * width))

                    # Crop the image around the detected body
                    cropped_image = frame[y_min:y_max, x_min:x_max]

                    # Resize the cropped image
                    resized_cropped_image = cv2.resize(cropped_image, resize_dimensions)

                    # Save the resized cropped image as an image file
                    frame_filename = f"frame_{frame_number:04d}.jpg"
                    frame_path = os.path.join(output_video_folder, frame_filename)
                    cv2.imwrite(frame_path, resized_cropped_image)

        # Release the video capture object
        cap.release()

        print(f"{num_frames_to_select} frames randomly selected, cropped, and resized for video: {video_file}.")

        end_time = time.time()
        difference = end_time - start_time

        print(f"Time Taken: {difference} seconds")
        print('\n')

# Example usage
source_folder = "/content/drive/MyDrive/latest_dataset"
destination_folder = "/content/drive/MyDrive/sequence_train_data"
num_frames_to_select = 50
resize_dimensions = (224, 224)

process_videos(source_folder, destination_folder, num_frames_to_select, resize_dimensions)


Video: video197.mp4
FPS: 30
Total Frames: 84
50 frames randomly selected, cropped, and resized for video: video197.mp4.
Time Taken: 68.88037824630737 seconds


Video: video209.mp4
FPS: 120
Total Frames: 163
50 frames randomly selected, cropped, and resized for video: video209.mp4.
Time Taken: 74.685231924057 seconds


Video: video218.mp4
FPS: 30
Total Frames: 191
50 frames randomly selected, cropped, and resized for video: video218.mp4.
Time Taken: 68.01518321037292 seconds


Video: video207.mp4
FPS: 30
Total Frames: 154
50 frames randomly selected, cropped, and resized for video: video207.mp4.
Time Taken: 64.02729678153992 seconds


Video: video198.mp4
FPS: 30
Total Frames: 170
50 frames randomly selected, cropped, and resized for video: video198.mp4.
Time Taken: 20.313334465026855 seconds


Video: video210.mp4
FPS: 30
Total Frames: 168
50 frames randomly selected, cropped, and resized for video: video210.mp4.
Time Taken: 65.44027543067932 seconds


Video: video193.mp4
FPS: 30
Total F

In [ ]:
# def count_unique_words(file_path):
#     with open(file_path, 'r') as file:
#         lines = file.readlines()

#     # Initialize a set to store unique words
#     unique_words = set()

#     # Iterate over each line (caption)
#     for line in lines:
#         # Split the line by spaces and ignore the first word (video identifier)
#         words = line.strip().split()[1:]

#         # Add unique words from the caption to the set
#         unique_words.update(words)

#     return len(unique_words)

# # Example usage
# file_path = '/content/drive/MyDrive/latest_caption.txt'  # Replace with the actual path to your text file

# # Count the total number of unique words in the file
# total_unique_words = count_unique_words(file_path)

# # Print the total number of unique words
# print(f"Total Unique Words: {total_unique_words}")


Total Unique Words: 394


# To fetch frames if the video are save in different category folder

In [ ]:
# import os
# import cv2
# import random
# import shutil

# def process_videos_in_categories(parent_folder, destination_parent_folder, num_frames_to_select=50, resize_dimensions=(224, 224)):
#     categories = [folder for folder in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, folder))]

#     for category in categories:
#         category_folder = os.path.join(parent_folder, category)
#         destination_category_folder = os.path.join(destination_parent_folder, category)
#         os.makedirs(destination_category_folder, exist_ok=True)

#         video_files = [file for file in os.listdir(category_folder) if file.endswith('.mp4')]

#         for video_file in video_files:
#             video_name = os.path.splitext(video_file)[0]
#             video_folder = os.path.join(destination_category_folder, video_name)
#             os.makedirs(video_folder, exist_ok=True)

#             video_path = os.path.join(category_folder, video_file)
#             cap = cv2.VideoCapture(video_path)
#             total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#             selected_frames = random.sample(range(total_frames), min(num_frames_to_select, total_frames))

#             for frame_number in selected_frames:
#                 cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
#                 success, frame = cap.read()

#                 if success:
#                     resized_frame = cv2.resize(frame, resize_dimensions)
#                     frame_filename = f"{video_name}_frame_{frame_number:04d}.jpg"
#                     frame_path = os.path.join(video_folder, frame_filename)
#                     cv2.imwrite(frame_path, resized_frame)

#             cap.release()

# parent_folder = "/path/to/parent_folder"
# destination_parent_folder = "/path/to/destination_parent_folder"
# num_frames_to_select = 50
# resize_dimensions = (224, 224)

# process_videos_in_categories(parent_folder, destination_parent_folder, num_frames_to_select, resize_dimensions)